In [28]:
import h5py
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm


# 设置文件路径并得到数据集
train_data_file = "../CNN/data/train_data/"
test_data_file = "../CNN/data/test_data/"
# model_path   torch.save(model.state_dict(), '../CNN/Cmodel_weights.pth')
model_path = "../CNN/"

# 超参数
learning_rate = 1e-3  # 1e-4
weight_decay = 1e-4
batch_size = 32
n_epochs = 100
best_acc= 0
#提取数据
def data_image_label(mode):  #shuffle
        if mode == 'train':
            for i in range(1, 9):
                date_path = f"{train_data_file}" + "traindata" + f"{i}" + ".mat"

                if i != 1:
                    data_image = np.array(h5py.File(date_path)["video"], dtype=np.float32)
                    data_label = np.array(h5py.File(date_path)["label"], dtype=np.float32)
                    data_image=np.expand_dims(data_image,axis=2)  #增加维度
                    video_image=np.zeros(shape=(len(data_image)-20+1,3,20,32,32),dtype=np.float32)
                    for i in range(len(data_image)-20+1):
                        for j  in range(20):
                            if j != 0:
                                video_image_20=np.append(video_image_20,data_image[i+j,:,:,:,:],axis=1)
                            else:
                                video_image_20=data_image[i+j,:,:,:,:]
                        video_image_20=np.expand_dims(video_image_20,axis=0)  #增加维度
                        video_image[i,:,:,:,:]=video_image_20
                    # print(f"video_image.size:{video_image.shape}")
                    output_video_image=np.append(output_video_image,video_image,axis=0)
                    output_video_label=np.append(output_video_label,data_label[0:len(data_image)-20+1,:],axis=0)

                else:
                    data_image = np.array(h5py.File(date_path)["video"], dtype=np.float32)
                    data_label = np.array(h5py.File(date_path)["label"], dtype=np.float32)
                    data_image=np.expand_dims(data_image,axis=2)  #增加维度
                    video_image=np.empty(shape=(len(data_image)-20+1,3,20,32,32),dtype=np.float32)
                    for i in range(len(data_image)-20+1):
                        for j  in range(20):
                            if j != 0:
                                video_image_20=np.append(video_image_20,data_image[i+j,:,:,:,:],axis=1)
                            else:
                                video_image_20=data_image[i+j,:,:,:,:]
                        video_image_20=np.expand_dims(video_image_20,axis=0)  #增加维度
                        video_image[i,:,:,:,:]=video_image_20
                    output_video_image=video_image
                    output_video_label=data_label[0:len(data_image)-20+1,:]
                    # print(f"video_image.size:{video_image.shape}")
            image_tensor = torch.tensor(output_video_image, dtype=torch.float32)
            label_tensor = torch.tensor(output_video_label.squeeze(), dtype=torch.long)
            real_len = len(image_tensor)
            print(image_tensor.shape, label_tensor.shape)
            return image_tensor,label_tensor,real_len
        elif mode == "test":
            for i in range(1, 5):
                date_path = f"{test_data_file}" + "testdata" + f"{i}" + "_label.mat"
                if i != 1:
                    data_image = np.array(h5py.File(date_path)["video"], dtype=np.float32)
                    data_label = np.array(h5py.File(date_path)["label"], dtype=np.float32)
                    data_image=np.expand_dims(data_image,axis=2)  #增加维度
                    video_image=np.zeros(shape=(len(data_image)-20+1,3,20,32,32),dtype=np.float32)
                    for i in range(len(data_image)-20+1):
                        for j  in range(20):
                            if j != 0:
                                video_image_20=np.append(video_image_20,data_image[i+j,:,:,:,:],axis=1)
                            else:
                                video_image_20=data_image[i+j,:,:,:,:]
                        video_image_20=np.expand_dims(video_image_20,axis=0)  #增加维度
                        video_image[i,:,:,:,:]=video_image_20
                    # print(f"video_image.size:{video_image.shape}")
                    output_video_image=np.append(output_video_image,video_image,axis=0)
                    output_video_label=np.append(output_video_label,data_label[0:len(data_image)-20+1,:],axis=0)
                else:
                    data_image = np.array(h5py.File(date_path)["video"], dtype=np.float32)
                    data_label = np.array(h5py.File(date_path)["label"], dtype=np.float32)
                    data_image=np.expand_dims(data_image,axis=2)  #增加维度
                    video_image=np.empty(shape=(len(data_image)-20+1,3,20,32,32),dtype=np.float32)
                    for i in range(len(data_image)-20+1):
                        for j  in range(20):
                            if j != 0:
                                video_image_20=np.append(video_image_20,data_image[i+j,:,:,:,:],axis=1)
                            else:
                                video_image_20=data_image[i+j,:,:,:,:]
                        video_image_20=np.expand_dims(video_image_20,axis=0)  #增加维度
                        video_image[i,:,:,:,:]=video_image_20
                    output_video_image=video_image
                    output_video_label=data_label[0:len(data_image)-20+1,:]
                    # print(f"video_image.size:{video_image.shape}")
            image_tensor = torch.tensor(output_video_image, dtype=torch.float32)
            label_tensor = torch.tensor(output_video_label.squeeze(), dtype=torch.long)
            real_len = len(image_tensor)
            print(image_tensor.shape, label_tensor.shape)
            return image_tensor,label_tensor,real_len
train_image_tensor,train_label_tensor,train_real_len=data_image_label('train')
test_image_tensor,test_label_tensor,test_real_len=data_image_label('test')

torch.Size([21412, 3, 20, 32, 32]) torch.Size([21412])
torch.Size([2932, 3, 20, 32, 32]) torch.Size([2932])


In [29]:
class SAPI_FMF_Dataset(Dataset):
    def __init__(self,image_data,label_data,data_len):
        """
        Args:
            file_path (string): 图像文件 路径
            mode (string): 训练模式还是测试模式
            valid_ratio (float): 验证集比例
        """
        self.image_tensor=image_data
        self.label_tensor=label_data
        self.real_len=data_len
        print('Finished reading the set of Leaves Dataset ({} samples found)'.format(self.real_len))
    def __getitem__(self, index):
        # 从 image_arr中得到索引对应的文件名
        return self.image_tensor[index, :, :, :], self.label_tensor[index]
    def __len__(self):
        return self.real_len  # self.real_len = len(self.image_arr) 返回的是训练/验证/测试/图像的数量
train_dataset = SAPI_FMF_Dataset(train_image_tensor,train_label_tensor,train_real_len)
test_dataset = SAPI_FMF_Dataset( test_image_tensor,test_label_tensor,test_real_len)
# 定义data loader
train_Dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True,drop_last=True)  # 打开乱序  Falsenum_workers=0)
test_Dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False,drop_last=False)

Finished reading the set of Leaves Dataset (21412 samples found)
Finished reading the set of Leaves Dataset (2932 samples found)


In [6]:
# GPU计算
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'
device = get_device()
print(device)

cpu


In [1]:
#3维残差网络
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResNet10_3D(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet10_3D, self).__init__()
        self.conv1 = nn.Conv3d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(64, 64, 2)
        self.layer2 = self._make_layer(64, 128, 2, stride=2)
        self.layer3 = self._make_layer(128, 256, 2, stride=2)
        self.layer4 = self._make_layer(256, 512, 2, stride=2)
        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Sequential(
            nn.Linear(512, num_classes),
            nn.Dropout(0.5)
        )

    def _make_layer(self, inplanes, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv3d(inplanes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm3d(planes)
            )
        layers = [ResBlock(inplanes, planes, stride, downsample)]
        inplanes = planes
        for _ in range(1, blocks):
            layers.append(ResBlock(inplanes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


class ResBlock(nn.Module):
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv3d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv3d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out
model=ResNet10_3D()

C:\Users\dell\.conda\envs\ML1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
from torch import  optim
model.to(device)
model.device = device
optimizer = optim.AdamW(model.parameters(), lr=1e-4,
                        weight_decay=weight_decay)  # lr=learning_rate,weight_decay= weight_decay
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)
print(model.parameters)

<bound method Module.parameters of AlexNet3D(
  (conv1): Conv3d(3, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (pool1): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(64, 192, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn2): BatchNorm3d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv3d(192, 384, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn3): BatchNorm3d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv3d(384, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn4): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv3d(256, 256, ke

In [ ]:
#训练
from torch import  optim
model.to(device)
model.device = device
optimizer = optim.AdamW(model.parameters(), lr=1e-4,
                        weight_decay=weight_decay)  # lr=learning_rate,weight_decay= weight_decay
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)
print(model.parameters)
# train
# 对于分类任务，我们使用交叉熵作为性能的度量。
criterion = nn.CrossEntropyLoss()
for epoch in (range(n_epochs)):
    # 记录训练信息
    model.train()
    train_loss = []
    train_accs = []
    for batch in tqdm(train_Dataloader):
        # ---------- Training ----------  以下是训练模型
        imgs, labels = batch
        # print(imgs.shape)

        logits = model(imgs)  # 将图形数据带入模型计算预测
        # 计算交叉熵损失
        loss = criterion(logits, labels)
        # 清除上一步中存储在参数中的梯度。
        optimizer.zero_grad()
        # 计算参数的梯度。
        loss.backward()
        # 用计算的梯度更新参数。
        optimizer.step()

        # 计算当前批次的精度。   第一维度是0，第二维度是1
        acc = (logits.argmax(dim=-1) == labels).float().mean()
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
        # 记录损失和准确度
        train_loss.append(loss.item())
        train_accs.append(acc)

    # 训练集的平均损失和精度是记录值的平均值。
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)
    # 更新学习率
    scheduler.step(train_loss)
    print("第%d个epoch的学习率：%f" % (epoch + 1, optimizer.param_groups[0]['lr']))

    # 打印信息
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
# ---------- test ----------
    model.eval()
    test_loss = []
    test_accs = []
    for batch in tqdm(test_Dataloader):
        imgs, labels = batch

        # 不需要梯度验证.
        # Using  torch.no_grad()  accelerates the forward process.
        with torch.no_grad():
            logits = model(imgs.to(device))

        # 我们仍然可以计算损失（但不能计算梯度）。
        loss = criterion(logits, labels.to(device))

        # 计算当前批次的精度。
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # 记录损失和准确性
        test_loss.append(loss.item())
        test_accs.append(acc)

    # 整个验证集的平均损失和准确度是记录值的平均值
    test_loss = sum(test_loss) / len(test_loss)
    test_accs = sum(test_accs) / len(test_accs)

    # 打印信息.
    print(f"[ test ]  loss = {test_loss:.5f}, acc = {test_accs:.5f}")
  # 如果模型改进了，在这个时间点保存一个检查点
    if test_accs > best_acc:
        best_acc = test_accs
        path=model_path+"model_weights_3d"+f"ResNet10_"+f'{best_acc}'+".pth"
        torch.save(model.state_dict(), path)
        print('saving model with acc {:.3f}'.format(best_acc))

In [14]:
#预测
import pandas as pd
test_data_file = "../CNN/data/test_data/"
def test_data(i):
    date_path = f"{test_data_file}" + "testdata" + f"{i}" + ".mat"
    data_image = np.array(h5py.File(date_path)["video"], dtype=np.float32)
    data_image=np.expand_dims(data_image,axis=2)  #增加维度
    video_image=np.empty(shape=(len(data_image)-20+1,3,20,32,32),dtype=np.float32)
    for i in range(len(data_image)-20+1):
        for j  in range(20):
            if j != 0:
                video_image_20=np.append(video_image_20,data_image[i+j,:,:,:,:],axis=1)
            else:
                video_image_20=data_image[i+j,:,:,:,:]
        video_image_20=np.expand_dims(video_image_20,axis=0)  #增加维度
        video_image[i,:,:,:,:]=video_image_20
    output_video_image=video_image
    # print(f"video_image.size:{video_image.shape}")
    image_tensor = torch.tensor(output_video_image, dtype=torch.float32)
    real_len = len(image_tensor)
    print(image_tensor.shape)
    return image_tensor,real_len

class test_result_Dataset(Dataset):
    def __init__(self,image_data,data_len):
        """
        Args:
            file_path (string): 图像文件 路径
        """
        self.image_tensor=image_data
        self.real_len=data_len
        print('Finished reading the  Dataset ({} samples found)'.format(self.real_len))
    def __getitem__(self, index):
        # 从 image_arr中得到索引对应的文件名
        return self.image_tensor[index, :, :, :]
    def __len__(self):
        return self.real_len

model=ResNet10_3D()
model_path="../CNN/model_weights_3dResNet10_0.9430027008056641.pth"
model.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
model.eval()
batch_size=32
for i in range(5,9):
    test_data_image,test_data_len=test_data(i)
    test_dataset = test_result_Dataset(test_data_image,test_data_len)
    test_Dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False,drop_last=False)
    predictions = []
    for batch in tqdm(test_Dataloader):
        imgs = batch
        with torch.no_grad():
            logits = model(imgs.to(device))
        predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())
    excel_path="../CNN/data/test_data/testdata"+f"{i}"+"_cls.xlsx"
    index=np.array(range(1,test_data_len+1)).tolist()
    data_toexcel={"index":index,"result":predictions}
    data_toexcel = pd.DataFrame(data_toexcel)
    data_toexcel.to_excel(excel_path, sheet_name="sheet1",index=False)
    print(predictions)

torch.Size([582, 3, 20, 32, 32])
Finished reading the  Dataset (582 samples found)


100%|██████████| 19/19 [00:10<00:00,  1.79it/s]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

100%|██████████| 31/31 [00:19<00:00,  1.63it/s]


[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

100%|██████████| 16/16 [00:10<00:00,  1.56it/s]


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

100%|██████████| 19/19 [00:12<00:00,  1.58it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 